# Import

In [1]:
# import moduels
import h5py
import numpy as np
import matplotlib.pylab as plt
import math
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

# settings

In [2]:
# access the data
# this part have to be replaced when the images from the neuronal networks are available
dset = h5py.File("dataset_test.h5","r")

#Access to the input data
RGB = dset["RGB"]
NIR = dset["NIR"]
print("Thes shape of the images is:")
print(RGB.shape)


# get size and numbers of the dataset
# number of images
n_images = RGB.shape[0]

# imagesize x_direction
sx_image = RGB.shape[1]
#imageszize y_direction
sy_image = RGB.shape[2]

Thes shape of the images is:
(3, 10980, 10980, 3)


In [3]:
# set number of images in x and y direction

# number of images in x direction
nx = 4
#number of images in y direction 
ny = 4

# size of patch
# alle sind quadratisch
s_patch = 256

# numbers of categories in the pretrained network Pascal Voc --> 21
n_cat = 21

In [4]:
# compute the coordinates of all top left pixels of each patch
# is for each image the same

x_max = sx_image-s_patch
y_max = sy_image-s_patch 

stepsize_x = math.floor(x_max/nx)
stepsize_y = math.floor(y_max/ny)

#create arrays with the corrdinates
x_coordinates = np.arange(0,x_max,stepsize_x)
y_coordinates = np.arange(0,y_max,stepsize_y)

# create grid
xx , yy = np.meshgrid(x_coordinates,y_coordinates)

# Network part

In [5]:
# import networks modules
from torchvision import models
import torchvision.transforms as T
import torch
# dir(models)

In [6]:
# select model

# more models are avialable at: https://pytorch.org/vision/stable/models.html
# https://learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/

#FCN ResNet50
model_rn50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
#model_rn50 = model_rn50.eval()

#FCN ResNet101
model_rn101 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
#model_rn101 = model_rn101.eval()

#other models that could be used

#DeeplabV3 ResNet50
#model_dl50 = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
#model_dl50 = model_dl50.eval()

#DeeplabV3 ResNet101
#model_dl101 = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True)
#model_dl101 = model_dl101.eval()


Using cache found in C:\Users\chrig/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\chrig/.cache\torch\hub\pytorch_vision_v0.10.0


In [7]:
# Specify image transformation
# function to prepare the input image into the correct form (for all segementation models of torchvision the same normalistation and standard deviation can be used!)
# more information: https://learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/
# or https://pytorch.org/vision/stable/models.html
transform = T.Compose([T.Resize(s_patch),
                 T.CenterCrop(s_patch),
                 T.ToTensor(), 
                 T.Normalize(mean = [0.485, 0.456, 0.406], 
                             std = [0.229, 0.224, 0.225])])

In [8]:
# creat function for model application
def get_segment_labels(image, model):
    # transform the image to tensor and load into computation device
    image = transform(image)
    image = image.unsqueeze(0) # add a batch dimension
    outputs = model(image)
    # uncomment the following lines for more info
    # print(type(outputs))
    # print(outputs['out'].shape)
    # print(outputs)
    return outputs

In [10]:
# run over each image patch and save data
zero_vector = np.zeros([1,21])
for img_n in range(3):
    for i in range(len(yy)):
        for j in range(len(xx)):
            temp_img = Image.fromarray(RGB[img_n,xx[i][j]:xx[i][j]+s_patch, yy[i][j]:yy[i][j]+s_patch,:3])
            out = get_segment_labels(temp_img,model_rn50) # try different models
            # print(out)
            
            temp = out.detach().numpy()
            temp1 = temp[0]
            temp2 = temp1.reshape(np.square(s_patch),n_cat)
            zero_vector[0][0] = xx[i][j]
            zero_vector[0][1] = yy[i][j]
            zero_vector[0][2] = img_n
            temp3 = np.append(temp2,zero_vector,axis =0)

            # save array
            np.save('./output_network_model_2/'+str(img_n)+'_'+str(xx[i][j])+'_'+str(yy[i][j])+'.npy',temp3)
            

ValueError: cannot reshape array of size 1000 into shape (65536,21)